In [46]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import brian2 as br
from time import time
from Reservoir.reservoir import Reservoir
import torchvision.datasets as datasets
from sklearn.preprocessing import normalize as norm
from sklearn.model_selection import train_test_split

In [47]:
def tr_res(image):
    return image.resize((10, 10))

In [48]:
dat = datasets.MNIST(root='./data', train=True, download=True, transform=tr_res)

9913344it [05:05, 32480.05it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


29696it [00:00, 177974.53it/s]                           


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw


1649664it [00:49, 33380.09it/s]                             


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


5120it [00:00, ?it/s]                   
WARNING    C:\Users\hi\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
 [py.warnings]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


In [3]:
y=pd.Series(y)
y=np.array(y.replace({11:0,13:1,211:2,321:3,2212:4}))
particle_name={0:'electron',1:'muon',2:'pion',3:'kaon',4:'proton'}

In [4]:
def show_images(X,y,n):
    fig,axes=plt.subplots(int(np.floor(n/4)),4,figsize=(14,int(np.floor(n/4))*8))
    
    #sampling examples from data
    for i in range(n):
        sample_id=np.random.choice([i for i in range(len(os.listdir(directory)))])
        
        #plotting image with particle name:
        plt.subplot(np.floor(n/4),4,i+1)
        plt.imshow(X[i],cmap='gray')
        plt.title(f'{particle_name[y[i]]}')
        plt.colorbar(label='intensity',
                     fraction=0.4,orientation='horizontal') 
        plt.axis('off')
        
    plt.tight_layout()
    plt.grid('off')
    plt.show()
    

In [5]:
pd.Series([particle_name[i] for i in y]).value_counts()

pion        906047
kaon        154323
proton      111730
electron      3138
muon          1237
dtype: int64

In [6]:
X.shape

(1176475, 10, 10)

In [7]:
X_two_c = X[y <= 1]
y_two_c = y[y <= 1]


In [24]:
X_two_c = X_two_c.reshape((X_two_c.shape[0], 1, X_two_c.shape[1], X_two_c.shape[2]))

In [25]:
x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.3,shuffle=True)

In [26]:
import torch
from torch import dtype, max_pool2d, nn

In [27]:
class CNN_classifier(nn.Module):
    def __init__(self, output_size, device="cpu"):
        super(CNN_classifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.proj = nn.Sequential(
            nn.Linear(100, 25), nn.ReLU(),
            nn.Linear(25, 25), nn.ReLU(),
            nn.Linear(25, output_size), nn.Softmax()
        )
    def forward(self, im):
        x = self.conv(im)
        return self.proj(x) 

In [28]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, x, labels):
        'Initialization'
        self.labels = labels
        self.x = x

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.x)

  def __getitem__(self, index):
        'Generates one sample of data'

        # Load data and get label
        X = self.x[index]
        y = self.labels[index]

        return X, y

In [29]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True}

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

training_set = Dataset(x_train, y_train)
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = Dataset(x_val, y_val)
validation_generator = torch.utils.data.DataLoader(validation_set, **params)


In [ ]:
params = {'batch_size': 64,
          'shuffle': True}

training_generator = torch.utils.data.DataLoader(dat, **params)

In [43]:
lr = 0.0001 
max_epochs = 300
print(f"Using {device}")
model = CNN_classifier(output_size=2)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
crit = nn.NLLLoss()
train_losses = []
val_losses = []
t0 = time()
for epoch in range(max_epochs):
    train_losses_epoch = []
    val_losses_epoch = []
    # Training
    i = 0
    for local_batch, local_labels in training_generator:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        pred = model(local_batch.float())
        loss = crit(pred.float(), local_labels)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses_epoch.append(loss.item())
    '''i = 0
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in validation_generator:
            # Transfer to GPU
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            pred = model(local_batch.float())
            loss = crit(pred.float(), local_labels)
            val_losses_epoch.append(loss.item())'''
    train_losses.append(np.mean(train_losses_epoch))
    #val_losses.append(np.mean(val_losses_epoch))
    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1}/{max_epochs}, train loss {train_losses[-1]}, val loss {val_losses[-1]}, time {(time() - t0)/60} ETA {(time() - t0)*max_epochs/(epoch+1)/60}")

Using cuda:0
Epoch 10/300, train loss -0.7055921417971452, val loss -0.7404870220593044, time 0.029320927460988362 ETA 0.8796278238296509
Epoch 20/300, train loss -0.7066203604141871, val loss -0.7405978242556254, time 0.05521325270334879 ETA 0.8281987905502319
Epoch 30/300, train loss -0.7063559467593828, val loss -0.7428037410690671, time 0.08170023759206137 ETA 0.8170023759206136
Epoch 40/300, train loss -0.7065667286515236, val loss -0.7435374089649746, time 0.10732747713724772 ETA 0.8049560785293579
Epoch 50/300, train loss -0.7066398821771145, val loss -0.7428539679163978, time 0.13435569206873577 ETA 0.8061341524124146
Epoch 60/300, train loss -0.7068876139819622, val loss -0.7442595987092882, time 0.1608270208040873 ETA 0.8041351040204366
Epoch 70/300, train loss -0.7066502397259077, val loss -0.7428663231077648, time 0.18633561531702678 ETA 0.7985812085015433
Epoch 80/300, train loss -0.7066525717576345, val loss -0.7428691273643857, time 0.21303356091181438 ETA 0.798875853419